In [1]:
# use with local installs that don't have nbdev
import sys

sys.path.insert(0, "../../")

In [2]:
# pip install nbdev

In [3]:
from selenium.webdriver.common.by import By
import datacrew.crawler.crawler as dcc
import datacrew.crawler.article as dca

BASE_URL = "https://domo-support.domo.com"

IMG_BASE_URL = "https://domo-support.domo.com/servlet/rtaImage"

OUTPUT_FOLDER = "../../raw_kb"

driver = dcc.driversetup(is_headless=False)

#### Sample implementation of test_article

In [4]:
TEST_ARTICLE_URL = "https://domo-support.domo.com/s/article/360043429913"

test_article = dca.Article_KB(
    url=TEST_ARTICLE_URL, driver=driver, base_url=BASE_URL)

test_article.url_ls

💤 retrieving https://domo-support.domo.com/s/article/360043429913 💤


['https://domo-support.domo.com/s/knowledge-base',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC',
 'https://domo-support.domo.com/s/article/360042924034',
 'https://domo-support.domo.com/s/article/360043429933',
 'https://domo-support.domo.com/s/article/360043429953',
 'https://domo-support.domo.com/s/article/360042924094',
 'https://domo-support.domo.com/s/article/360043429993',
 'https://domo-support.domo.com/s/article/360043428753',
 'https://domo-support.domo.com/s/article/360043429913',
 'https://domo-support.domo.com/s/article/360042925494',
 'https://domo-support.domo.com/s/article/360043931814']

#### sample implementation of test_category

In [5]:
TEST_TOPIC_URL = 'https://domo-support.domo.com/s/topic/0TO5w000000ZamsGAC'

test_category = dca.Article_Category(
    url=TEST_TOPIC_URL, driver=driver, base_url=BASE_URL, is_child_recursive =True)

# for key in test_category.__dict__.keys():
#     if key in ['soup','article_soup']:
#         continue

#     print({key: getattr(test_category, key)})

test_category.url_ls

💤 retrieving https://domo-support.domo.com/s/topic/0TO5w000000ZamsGAC 💤
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZanlGAC, 0TO5w000000ZanlGAC
💤 retrieving https://domo-support.domo.com/s/topic/0TO5w000000ZanlGAC 💤
❤️ child url - https://domo-support.domo.com/s/article/360042922634, 360042922634
❤️ child url - https://domo-support.domo.com/s/article/360043442453, 360043442453
❤️ child url - https://domo-support.domo.com/s/article/360061981173, 360061981173
❤️ child url - https://domo-support.domo.com/s/article/360042922834, 360042922834
❤️ child url - https://domo-support.domo.com/s/article/360043427433, 360043427433
❤️ child url - https://domo-support.domo.com/s/article/360042922754, 360042922754
❤️ child url - https://domo-support.domo.com/s/article/360043427413, 360043427413
❤️ child url - https://domo-support.domo.com/s/article/360043877854, 360043877854
❤️ child url - https://domo-support.domo.com/s/article/4409542407063, 4409542407063
❤️ child url - https://d

['https://domo-support.domo.com/s/knowledge-base',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanlGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanxGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanCGAS',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanmGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000Zao5GAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZaoIGAS',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZaoLGAS',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanBGAS',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZankGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZancGAC',
 'https://domo-support.domo.com/s/article/360042922634',
 'https://domo-support.domo.com/s/article/360043442453',
 'https://domo-support.domo.com/s/article/360061981173',
 'https://domo-support.domo.com/s/article/360042922834',
 'https://domo-support.domo.com/s/article/36004342

#### sample implementation of test_home

In [6]:
TEST_HOME_URL = "https://domo-support.domo.com/s/knowledge-base"

test_home = dca.Article_KB_Home(
    url=TEST_HOME_URL, driver=driver, base_url=BASE_URL)

for key in test_home.__dict__.keys():
    if key in ['soup','article_soup']:
        continue

    print({key: getattr(test_home, key)})

💤 retrieving https://domo-support.domo.com/s/knowledge-base 💤
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamsGAC, 0TO5w000000ZamsGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC, 0TO5w000000ZammGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamzGAC, 0TO5w000000ZamzGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC, 0TO5w000000ZamoGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamnGAC, 0TO5w000000ZamnGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamlGAC, 0TO5w000000ZamlGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamqGAC, 0TO5w000000ZamqGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000Zan0GAC, 0TO5w000000Zan0GAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000Zan2GAC, 0TO5w000000Zan2GAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZampGAC, 0TO5w000000ZampGAC
❤️ child url - htt

In [7]:
# def get_child_category(article : dca.Article, res = [] ):
    
#     if isinstance(article, dca.Article) and article not in res:
#         res.append(article)

#     for article in article.child_category_ls:
        
#         if not article:
#             continue
        
#         if isinstance(article, dca.Article):
#             if article not in res:
#                 res.append(article)

#             get_child_category(article, res)
    
#     return list(res)

# # get_child_category(test_category)


# def get_child_category_url_ls(article: dca.Article_Category, url_ls=[], tested_article =[]):
    
#     if article.url not in url_ls and article not in tested_article:
#         url_ls.append(article.url)
#         tested_article.append(article)


#     for child_article in article.child_category_ls:
        
#         if not child_article or child_article in tested_article:
#             continue
        
#         if child_article.url not in url_ls:
#             url_ls.append(child_article.url)
        
#         tested_article.append(child_article)

#         get_child_category_url_ls(child_article, url_ls, tested_article)

#     return url_ls


# # test_category.child_category_ls
# test_category.child_category_ls


# Utility Functions

## string manipulation


In [8]:
import re

def convert_to_snake(text_str):
    """converts 'snake_case_str' to 'snakeCaseStr'"""

    return text_str.replace(" ", "_").lower()


def clean_url_name(path_name):
    valid_chars = r"[^a-zA-Z0-9_]"

    return re.sub(valid_chars, "", path_name)


def get_id_from_url(url: str, url_match: str):
    """use url_match" to identify the id of an object"""
    return url.split(url_match)[1].split("/")[0]


## process html files


### download image

In [9]:
import requests
import pandas as pd


def download_img(image_url, image_path, debug_prn: bool = False):

    img_data = requests.get(image_url).content

    with open(image_path, "wb") as handler:
        if debug_prn:
            print(f"downloading {image_url} to {image_path}")
        handler.write(img_data)


def download_article_images(
    article: dca.Article,
    output_path: str,
    debug_prn: bool = False,
):
    image_ls = article.image_ls

    if not image_ls:
        return 

    # download images
    for img in image_ls:
        img_name = img.get("name")

        if not img_name:
            continue

        img_url = img.get("url")
        img_path = f"{output_path}/{img_name}"
        img_rel_path = img.get("relative_url")

        if debug_prn:
            print(
                f"downloading {img_url} to {img_path}.  replacing article with {img_rel_path} with {img_name}"
            )

        download_img(image_url=img_url, image_path=img_path)

        article.md_str = article.md_str.replace(img_rel_path, img_name)

    return image_ls

test_output_path = "../../raw_kb/article/adding_a_beast_mode_calculation_to_your_chart"

pd.DataFrame(download_article_images(
    article=test_article,
    output_path=test_output_path,
    debug_prn=False,
))

,url,relative_url,name
0,https://domo-support.domo.com/servlet/rtaImage...,/servlet/rtaImage?eid=ka05w000001248e&feoid=00...,Beast_Mode_Dialog.jpg
1,https://domo-support.domo.com/servlet/rtaImage...,/servlet/rtaImage?eid=ka05w000001248e&feoid=00...,
2,https://domo-support.domo.com/servlet/rtaImage...,/servlet/rtaImage?eid=ka05w000001248e&feoid=00...,Beast_Mode_Modal.png
3,https://domo-support.domo.com/servlet/rtaImage...,/servlet/rtaImage?eid=ka05w000001248e&feoid=00...,
4,https://domo-support.domo.com/servlet/rtaImage...,/servlet/rtaImage?eid=ka05w000001248e&feoid=00...,


### download html

In [10]:
import os


def create_output_folder(output_folder, title: str):
    output_path = os.path.join(
        output_folder, clean_url_name(convert_to_snake(title))
    )

    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path


def output_html(output_html_path, soup):
    with open(output_html_path, 'w', encoding='utf-8') as f:
        f.write(str(soup ).encode('utf-8'))


# handle category

In [11]:
import os
import pandas as pd
import datetime as dt

def update_listing(
        url: str,
        url_id: str,
        name: str,
        file_name: str,
        output_file_path:str,
        output_folder):

    output_file = f"{output_folder}/{file_name}"

    df = None
    if os.path.exists(output_file):
        df = pd.read_csv(output_file, index_col='id')

    else:
        columns = ['id','name','url', 'updated', 'output_file_path']
        df = pd.DataFrame(columns=columns).set_index('id')


    df.loc[url_id] = [name, url, dt.datetime.now(), output_file_path]

    df.to_csv(output_file)

    return df.loc[url_id]


update_listing(url=test_category.url,
                 url_id = test_category.url_id,
                 name = test_category.category,
                 file_name = 'category_mapping.csv',
               output_file_path='test.html',
                 output_folder=os.path.join(OUTPUT_FOLDER, 'category'))


name                                             Getting To Know Domo
url                 https://domo-support.domo.com/s/topic/0TO5w000...
updated                                    2023-04-16 20:23:21.754834
output_file_path                                            test.html
Name: 0TO5w000000ZamsGAC, dtype: object

In [12]:
from mdutils.mdutils import MdUtils
import pandas as pd


def process_category(article: dca.Article,
                    output_folder: str,
                    debug_prn: bool = False):

    output_path = create_output_folder(os.path.join(output_folder,'category'), article.category)

    output_html_path = os.path.join(output_path, 'index.html')
    output_html(output_html_path, article.soup)

    update_listing(url=article.url,
                 url_id = article.url_id,
                 name = article.category,
                 file_name = 'category_listing.csv',
                 output_file_path = output_html_path,
                 output_folder=OUTPUT_FOLDER)

    download_article_images(
        article=article,
        output_path= output_path)

process_category(test_category, output_folder=OUTPUT_FOLDER)


## handle article


In [13]:
from mdutils.mdutils import MdUtils
import pandas as pd


def process_article(article: dca.Article,
                    output_folder: str,
                    debug_prn: bool = False):

    output_path = create_output_folder(os.path.join(output_folder, 'article'), article.title)
    
    output_html_path = os.path.join(output_path, 'index.html')

    if debug_prn:
        print(output_html_path)

    output_html(output_html_path, article.soup)

    update_listing(url=article.url,
                url_id = article.url_id,
                name = article.title,
                file_name = 'article_listing.csv',
                output_file_path = output_html_path,
                output_folder=OUTPUT_FOLDER)

    download_article_images(
        article=article,
        output_path= output_path, debug_prn=  debug_prn)

    

process_article(test_article, output_folder=OUTPUT_FOLDER ,debug_prn = True)


../../raw_kb\article\adding_a_beast_mode_calculation_to_your_chart\index.html
downloading https://domo-support.domo.com/servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOEH to ../../raw_kb\article\adding_a_beast_mode_calculation_to_your_chart/Beast_Mode_Dialog.jpg.  replacing article with /servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOEH with Beast_Mode_Dialog.jpg
downloading https://domo-support.domo.com/servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOED to ../../raw_kb\article\adding_a_beast_mode_calculation_to_your_chart/Beast_Mode_Modal.png.  replacing article with /servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&refid=0EM5w000005vOED with Beast_Mode_Modal.png


In [19]:
import logging
import selenium.webdriver
import os
import time

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s", level=logging.INFO)


class Crawler:
    base_url: str
    output_folder: str

    urls_visited_ls: list[str]
    urls_to_vist_ls: list[str]
    urls_error_ls: list[str]

    path_to_visit: str
    path_errors : str
    path_visited: str

    driver: selenium.webdriver

    counter: int

    def __init__(
        self,
        urls_to_visit_ls: list[str] = None,
        base_url=None,
        output_folder="../../raw_kb/",
        is_fresh_start: bool = False
    ):
        self.base_url = base_url
        self.output_folder = output_folder

        self.counter = 0

        self.path_to_visit = os.path.join(
            self.output_folder, 'crawler_to_visit.csv')

        self.path_visited = os.path.join(
            self.output_folder, 'crawler_visited.csv')
        
        self.path_errors = os.path.join(self.output_folder, 'crawler_errors.csv')

        self.article_ls = []
        self.driver = dcc.driversetup(is_headless=False)

        if is_fresh_start:
            "✂️ deleting files"
            self._delete_file(self.path_to_visit)
            self._delete_file(self.path_visited)
            self._delete_file(self.path_errors)

        self.urls_visited_ls = []
        self.urls_to_visit_ls = []
        self.urls_error_ls =[]

        if not is_fresh_start:
            self.urls_visited_ls = self._read_file_ls(self.path_visited)

            urls_to_visit_ls += self._read_file_ls(self.path_errors, is_reverse = True)
            urls_to_visit_ls += self._read_file_ls(self.path_to_visit)
        
        print(urls_to_visit_ls)

        [self.add_url_to_visit(url) for url in list(set(urls_to_visit_ls))]

    @staticmethod
    def _delete_file(file_path):
        if os.path.exists(file_path):
            print(f'deleting {file_path}')
            os.remove(file_path)
        else:
            print(f"{file_path} cannot be deleted")

    @staticmethod
    def _read_file_ls(file_path, is_reverse: bool = False):
        try:
            file = open(file_path, '+r')

            if is_reverse:
                file.reverse()

            return [line.strip() for line in file]
        except:
            return []

    @staticmethod
    def _write_file_ls(file_path, data):

        file = open(file_path, 'w+', encoding="utf-8")

        for item in data:
            file.write(item+"\n")
        file.close()

    def add_url_to_visit(self, url, debug_prn: bool = False):
        if url not in self.urls_visited_ls and url not in self.urls_to_visit_ls:
            if debug_prn:
                print(f"adding url to list - {url}")

            self.urls_to_visit_ls.append(url)

    def crawl(self, url, debug_prn: bool = False):
        if debug_prn:
            print(f"starting crawl - {url}")

        article = None
        if '/s/topic/' in url:
            is_child_recursive = True
            article = dca.Article_Category(url=url,
                                           driver=driver,
                                           base_url=self.base_url,
                                           is_child_recursive=is_child_recursive
                                           )
        
            process_category(article=article, output_folder=self.output_folder)

        if '/s/article/' in url:
            # return
            article = dca.Article_KB(
                url=url, base_url=self.base_url, driver=driver)
                
            process_article(article=article, output_folder=self.output_folder)
        
        if '/s/knowledge-base' in url:
            article = dca.Article_KB_Home(
                url=url, base_url=self.base_url, driver=driver)
        
        time.sleep(1)

        if not article:
            return
        
        [self.add_url_to_visit(url=url, debug_prn=debug_prn) for url in article.url_ls]

        for url in article.url_ls:
            self.add_url_to_visit(url=url, debug_prn=debug_prn)



    def run(self, debug_prn: bool = False):
        while self.urls_to_visit_ls:
            url = self.urls_to_visit_ls.pop(0)

            logging.info(f"Crawling: {url}")

            try:
                self.crawl(url, debug_prn)
                self.urls_visited_ls.append(url)
                
            except Exception:
                logging.exception(f"Failed to crawl: {url}")
                self.urls_error_ls.append(url)
            finally:
                self.counter += 1
                
                if self.counter % 10 == 0:
                    self._write_file_ls(self.path_to_visit, self.urls_to_visit_ls)
                    self._write_file_ls(self.path_visited, self.urls_visited_ls)
                    self._write_file_ls(self.path_errors, self.urls_error_ls)

        print("done")
        return self


In [20]:
from pprint import pprint

crawler = Crawler(
    urls_to_visit_ls=[TEST_HOME_URL], 
    base_url=BASE_URL, 
    output_folder= OUTPUT_FOLDER, 
    is_fresh_start = False
)

pprint(crawler.__dict__)

crawler.run(debug_prn=False)

2023-04-17 11:39:49,596 INFO:Crawling: https://domo-support.domo.com/s/topic/0TO5w000000ZaoUGAS
2023-04-17 11:39:49,597 ERROR:Failed to crawl: https://domo-support.domo.com/s/topic/0TO5w000000ZaoUGAS
Traceback (most recent call last):
  File "C:\Users\jaewi\AppData\Local\Temp\ipykernel_34508\2425331802.py", line 150, in run
    self.crawl(url, debug_prn)
  File "C:\Users\jaewi\AppData\Local\Temp\ipykernel_34508\2425331802.py", line 112, in crawl
    article = dca.Article_Category(url=url,
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\GitHub\datacrew\nbs\projects\../..\datacrew\crawler\article.py", line 338, in __init__
    driver=None,
               ^
  File "c:\GitHub\datacrew\nbs\projects\../..\datacrew\crawler\crawler.py", line 113, in pagesource
    driver.get(url)
  File "C:\Users\jaewi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\selenium\webdriver\remote\webdriver.py", line 449, in get
    s

['https://domo-support.domo.com/s/knowledge-base', 'https://domo-support.domo.com/s/topic/0TO5w000000ZaoUGAS', 'https://domo-support.domo.com/s/topic/0TO5w000000ZanSGAS', 'https://domo-support.domo.com/s/topic/0TO5w000000Zao8GAC', 'https://domo-support.domo.com/s/topic/0TO5w000000Zao6GAC']
{'article_ls': [],
 'base_url': 'https://domo-support.domo.com',
 'counter': 0,
 'driver': <selenium.webdriver.chrome.webdriver.WebDriver (session="45a85e59f47c753aab1f7bed7efe54ce")>,
 'output_folder': '../../raw_kb',
 'path_errors': '../../raw_kb\\crawler_errors.csv',
 'path_to_visit': '../../raw_kb\\crawler_to_visit.csv',
 'path_visited': '../../raw_kb\\crawler_visited.csv',
 'urls_error_ls': [],
 'urls_to_visit_ls': ['https://domo-support.domo.com/s/topic/0TO5w000000ZaoUGAS',
                      'https://domo-support.domo.com/s/topic/0TO5w000000ZanSGAS',
                      'https://domo-support.domo.com/s/topic/0TO5w000000Zao8GAC',
                      'https://domo-support.domo.com/s/topic

In [16]:
import pandas as pd 
import os

def read_listing(
    output_folder,
    file_name: str,
    ):

    output_file = f"{output_folder}/{file_name}"

    return pd.read_csv(output_file, index_col='id')


def write_file_ls(file_path, data):

    file = open(file_path, 'w+', encoding="utf-8")

    for item in data:
        file.write(item+"\n")
    file.close()

# articles to visited
article_ls = list(read_listing(OUTPUT_FOLDER, 'article_listing.csv')['url'])
write_file_ls(os.path.join(OUTPUT_FOLDER, 'crawler_visited.csv'), data = article_ls)

#category to to_visit
category_ls = list(read_listing(OUTPUT_FOLDER, 'category_listing.csv')['url'])
write_file_ls(os.path.join(OUTPUT_FOLDER, 'crawler_to_visit.csv'), data = category_ls)